In [28]:
import numpy as np
import pandas as pd
import shutil
from roboflow import Roboflow
import os
import re

In [29]:
def clean_up(integral=False):
    if os.path.exists('GARBAGE-CLASSIFICATION-3-2'):
        shutil.rmtree('GARBAGE-CLASSIFICATION-3-2')
    if integral:
        if os.path.exists('data'):
            shutil.rmtree('data')
    else:
        for file in os.listdir('data'):
            file = os.path.join('data', file)
            if os.path.isfile(file):
                os.remove(file)

def move_files_recursively(src,dest='data',files=None):
    if not os.path.exists(dest):
        os.makedirs(dest)
    files_to_move = []
    if files is None:
        for root, dirs, files in os.walk(src):
            for file in files:
                if file.endswith('.jpg'):
                    files_to_move.append(os.path.join(root, file))
    else:
        interesting_files = list(set(files) & set(os.listdir(src)))
        for file in interesting_files:
            files_to_move.append(os.path.join(src, file))
    for file in files_to_move:
        shutil.move(file, dest)

In [30]:
clean_up(True)
rf = Roboflow(api_key="PYZngL70ijcagLdXpb6l")
project = rf.workspace("material-identification").project("garbage-classification-3")
dataset = project.version(2).download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to GARBAGE-CLASSIFICATION-3-2 in tensorflow:: 100%|██████████| 10472/10472 [00:00<00:00, 11291.35it/s]


In [31]:
train = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/train/_annotations.csv')
test = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/test/_annotations.csv')
val = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/valid/_annotations.csv')
combined_data = pd.concat([train, test, val], ignore_index=True)

In [32]:
move_files_recursively('GARBAGE-CLASSIFICATION-3-2')

In [33]:
map_classes = {
    'METAL': 0,
    'CARDBOARD': 0,
    'PLASTIC': 0,
    'PAPER': 1,
    'GLASS': 2,
    'BIODEGRADABLE': 3
}
def get_class(c):
    for key in map_classes.keys():
        if key==c:
            return map_classes.get(key)
    return c

def remove_until_char(input_string,remove_char,include_char=True):
    match = re.search(remove_char, input_string)
    if match:
        return input_string[match.start() + 1 if include_char else 0:]
    return input_string

def edit_row(row):
    # file_name = row['filename']
    classe = get_class(row['class'])
    # file_name_modified = f'{classe.lower()}{remove_until_char(file_name,remove_char="_",include_char=False)}'
    # row['filename'] = file_name_modified
    row['class'] = classe
    # if os.path.exists(f'data/{file_name}'):
    #     os.rename(f'data/{file_name}', f'data/{file_name_modified}')
    return row

for index, row in combined_data.iterrows():
    combined_data.loc[index] = edit_row(row)

classes = ['PAPER', 'GLASS', 'METAL_CARDBOARD_PLASTIC','BIODEGRADABLE']
combined_data['class'] = combined_data['class'].apply(get_class)

classes_occurences = combined_data['class'].value_counts()    

In [34]:
print(f'Dataframe shape: {combined_data.shape}')
combined_data = combined_data.sample(frac=1).reset_index(drop=True) #in place shuffle
print(f'Dataframe shape shuffled: {combined_data.shape}')
balanced_data = pd.DataFrame(columns=combined_data.columns)
for class_label in combined_data['class'].unique():
    class_samples = combined_data[combined_data['class'] == class_label].head(np.min(classes_occurences))
    balanced_data = pd.concat([balanced_data, class_samples], ignore_index=True)
print(f'Balanced dataframe shape: {balanced_data.shape}')
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
print(f'Balanced dataframe shape shuffled: {balanced_data.shape}')

Dataframe shape: (74090, 8)
Dataframe shape shuffled: (74090, 8)
Balanced dataframe shape: (17560, 8)
Balanced dataframe shape shuffled: (17560, 8)


In [35]:
ratios=[0.6, 0.25, 0.15]

def get_set_name():
    rand = np.random.uniform()
    if rand <= ratios[0]:
        set_name = 'TRAIN'
    elif rand <= ratios[0] + ratios[1]:
        set_name = 'TEST'
    else:
        set_name = 'VAL'
    return set_name

balanced_data['set'] = balanced_data.apply(lambda row: get_set_name(), axis=1) 

In [36]:
balanced_data.head().to_csv('data/annotations.csv', index=False)
balanced_data

,filename,width,height,class,xmin,ymin,xmax,ymax,set
0,paper1698_jpeg.rf.8f9209f8db6c8217bb3a22c65cf6...,416,416,1,0,1,415,415,VAL
1,paper305_jpg.rf.fe50060e4ff619ccfca8ece48dd093...,416,416,1,24,10,246,143,TRAIN
2,paper1174_jpg.rf.482d3389b117a5af7a4f1a9696701...,416,416,1,88,54,320,144,TEST
3,paper1230_jpg.rf.29a95d9839e17dd1e4b822fbb3600...,416,416,1,2,0,308,361,TEST
4,biodegradable2166_jpeg.rf.7b649a31ebc54f2d1df3...,416,416,3,289,221,307,243,TRAIN
...,...,...,...,...,...,...,...,...,...
17555,biodegradable2177_jpeg.rf.0f18b50efbdead9a6d21...,416,416,1,195,314,324,389,VAL
17556,biodegradable1397_jpg.rf.d83a0f42f37112a9511ae...,416,416,3,245,146,357,217,TRAIN
17557,paper1898_jpg.rf.df7551e7b72a8e74d10a791e376aa...,416,416,1,254,1,415,143,TRAIN
17558,paper1925_jpg.rf.4d86dd922ff6eb7308eec43ab556f...,416,416,1,158,22,401,412,TRAIN


In [ ]:
# set,filename,class,xmin_relative,ymin_relative,_,_,xmax_relative,ymax_relative,_,_
# VAL,paper1698_jpeg.rf.8f9209f8db6c8217bb3a22c65cf69f9c.jpg,1,0,0,,,1,1,,
# TRAIN,paper305_jpg.rf.fe50060e4ff619ccfca8ece48dd09355.jpg,1,.06,.02,,,.59,.34,,
# TEST,paper1174_jpg.rf.482d3389b117a5af7a4f1a9696701419.jpg,1,.21,.13,,,.77,.35,,
# TEST,paper1230_jpg.rf.29a95d9839e17dd1e4b822fbb360074e.jpg,1,0,0,,,.74,.87,,
# TRAIN,biodegradable2166_jpeg.rf.7b649a31ebc54f2d1df322d4ebe6975d.jpg,3,.69,.53,,,.74,.58,,